In [ ]:
from ratdata import data_manager as dm, process, ingest
import pathlib

In [ ]:
dm.db_connect('rat_data.db')
time_slices_file = 'data/mce_recordings/time_slices.pickle'
# Read time slices from the file
time_slices = ingest.read_file_slices(time_slices_file)

In [ ]:
recording_files = dm.RecordingFile.select()

In [ ]:
for i, f in enumerate(recording_files):
    fname = pathlib.Path(f.dirname) / f.filename
    recording = ingest.read_mce_matlab_file(str(fname))
    filename_stem = f.filename.split('.')[0]
    if filename_stem in time_slices:
        slice = time_slices[filename_stem]
        data, tt = process.trim_recording(recording.electrode_data.mean(0),
                                          int(1/recording.dt), slice['start'], slice['length'])
    else:
        data = recording.electrode_data.mean(0)
    recording_beta_power = process.compute_power_in_frequency_band(data, 10, 30, int(1/recording.dt))
    recording_total_power = process.compute_power_in_frequency_band(data, 1, 100, int(1/recording.dt))
    query = dm.RecordingPower.select(id).where(dm.RecordingPower.recording == dm.RecordingFile.select().where(dm.RecordingFile.filename == f))
    rp_id = dm.RecordingPower.replace(recording=f, beta_power=recording_beta_power, total_power=recording_total_power).execute()
    print('[%d/%d] %s: %d' % (i + 1, len(recording_files), f.filename, rp_id))
    